In [1]:
!pip install tenseal sentence_transformers

In [2]:
import tenseal as ts # pip install tenseal
from sentence_transformers import SentenceTransformer



f:\Mini-Conda\envs\test\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

f:\Mini-Conda\envs\test\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
sentences = [
   "The quick brown fox jumps over the lazy dog.",
   "I watched the sunset over the ocean.",
   "Artificial intelligence is transforming industries.",
   "The library was quiet and smelled like old books.",
   "He dreamed of traveling to distant galaxies.",
   "Innovation drives progress in the tech world.",
   "The chef prepared a delicious meal for the guests.",
   "Climate change poses a significant threat to global biodiversity.",
   "The athlete trained rigorously for the upcoming marathon.",
   "Music has the power to evoke deep emotional responses."
]

In [5]:
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.04393356  0.05893437  0.04817845 ...  0.05216283  0.05610655
   0.10206394]
 [ 0.02740664  0.04301418  0.03891345 ... -0.01506043 -0.08190969
   0.03566788]
 [ 0.03608516 -0.02447894  0.05895456 ... -0.0826278   0.07626031
  -0.02810952]
 ...
 [-0.04316908  0.12324496  0.09774677 ... -0.13736208 -0.05429132
  -0.02212724]
 [-0.01195204  0.08340724 -0.03579247 ... -0.00149685  0.00464321
  -0.00069312]
 [ 0.05495497 -0.01101494  0.09608353 ...  0.0507521   0.01033211
  -0.03917956]]


In [6]:
query_text = "artificial intelligence"
query_embedding = model.encode([query_text])
print(query_embedding)

[[-2.43510008e-02  1.67066492e-02  3.77205350e-02 -9.16334614e-03
  -3.05844937e-02 -1.70570984e-02  7.42098019e-02  4.57434356e-02
  -9.39853676e-03  9.90003720e-03 -5.70719643e-03  7.58130616e-03
   3.95796336e-02  1.52103230e-02 -8.32180381e-02  1.93441920e-02
  -2.19869111e-02 -3.32064331e-02 -1.81014091e-01 -1.30236104e-01
  -2.25592218e-03  1.33974878e-02 -2.42931824e-02 -3.69969197e-02
   2.02792673e-03  8.56798068e-02  4.72739432e-03 -3.41751822e-03
  -6.03483012e-03 -1.15787156e-01  6.68601841e-02 -1.86577030e-02
   8.78416747e-02 -7.42426515e-03 -9.35673565e-02  6.14010170e-02
  -8.11372325e-02  1.22457677e-02  3.97397652e-02 -2.58959970e-03
  -4.66312841e-02 -8.17788765e-02  3.95028554e-02  1.54695408e-02
   4.36929390e-02  1.03639163e-01 -5.84595054e-02  3.67365293e-02
  -5.27095497e-02  4.05691676e-02 -1.25884548e-01  6.46113092e-03
  -3.58352363e-02 -1.00229466e-02 -2.38813832e-02  4.59325314e-02
   1.45444460e-02  1.94852799e-02  2.84130704e-02 -5.51534817e-02
   2.42295

In [7]:
context = ts.context(
           ts.SCHEME_TYPE.CKKS,
           poly_modulus_degree = 8192,
           coeff_mod_bit_sizes = [60, 40, 40, 60]
         )


context.generate_galois_keys()
context.global_scale = 2**40

In [8]:
secret_context = context.serialize(save_secret_key = True)

In [9]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()

In [10]:
context = ts.context_from(secret_context)

In [11]:
enc_queryvec = ts.ckks_vector(context, query_embedding[0].tolist())

In [12]:
cosine_similarity_ranking = []
for i in range(len(sentences)):
    enc_sentence = ts.ckks_vector(context, embeddings[i].tolist())
    dot_product = enc_queryvec.dot(enc_sentence)
    cosine_similarity = dot_product.decrypt()[0]
    cosine_similarity_ranking.append({sentences[i]: abs(cosine_similarity)})

In [13]:
search_results = sorted(cosine_similarity_ranking, key=lambda x: list(x.values())[0], reverse=True)
for item in search_results:
    print(item)
    print()

{'Artificial intelligence is transforming industries.': 0.5606294297669284}

{'Innovation drives progress in the tech world.': 0.26706054339800056}

{'Music has the power to evoke deep emotional responses.': 0.1741896879622748}

{'The athlete trained rigorously for the upcoming marathon.': 0.11048240679011677}

{'The chef prepared a delicious meal for the guests.': 0.09010653878252728}

{'I watched the sunset over the ocean.': 0.07017185989003774}

{'He dreamed of traveling to distant galaxies.': 0.059552227540030124}

{'Climate change poses a significant threat to global biodiversity.': 0.042122459425155724}

{'The library was quiet and smelled like old books.': 0.03255208724006971}

{'The quick brown fox jumps over the lazy dog.': 0.008749775847558053}

